In [709]:
import numpy as np
import random
from typing import Literal

In [710]:
{'right':0,'up':90,'left':180,'down':270}

{'right': 0, 'up': 90, 'left': 180, 'down': 270}

In [711]:
class Rotator:
    def __init__(self, degree_to_rotate: Literal[0, 90, 180, 270]):
        # self.matrix = matrix
        self.degree_to_rotate = degree_to_rotate

    def __enter__(self):
        pass

    def rotate(self, matrix: np.ndarray)-> np.ndarray:
        return np.rot90(matrix, k=int(self.degree_to_rotate/90), axes=(1,0))
        

        
    def unrotate(self, matrix: np.ndarray)-> np.ndarray:
        return np.rot90(matrix, k=int(self.degree_to_rotate/90), axes=(0,1))
    
    # @property
    # def rotated(self):
    #     return self.rotate(self.matrix)
    
    # @property
    # def unrotated(self):
    #     return self.unrotate(self.rotated)
    
    def rot(self, deg: Literal[0, 90, 180, 270]):
        self.degree_to_rotate = deg
        return self
        
        
        




In [712]:
class Board:
    def __init__(self, n: int=4):
        self.length = n
        self.board = self.create_board(n)
        self.rotator = Rotator(0)

    def create_board(self, length):
        return np.zeros((length, length), dtype=np.int32)

    def reset(self):
        """ Born a board that either has two 2 or a 2 and 4. """
        self.clear()
        self._random_born()
        self._random_born(2 if np.random.random() < 0.5 else 4)

    def clear(self):
        self.board = self.create_board(self.length)

    @staticmethod
    def _push(vector: np.ndarray):
        """ Put all elements to the right side (0deg). """
        v = np.zeros_like(vector)
        non_zero = vector[vector != 0]
        if len(non_zero) == 0:
            return vector
        v[-len(non_zero):] = non_zero
        vector = v
        return vector

    @staticmethod
    def _join_reverse(vector: np.ndarray):
        """ Join the neighboring elements from left to right, e.g. (2,0,0,2)->(0,0,0,4) """
        previous = None
        previous_index = -1
        for i, current in enumerate(vector):
            if current == previous:
                vector[previous_index], vector[i] = 0, previous+current
                previous = None
                previous_index = -1

                continue
            if current != 0:
                previous = vector[i]  # Not current as we mutated the vector.
                previous_index = i
        return vector

    @staticmethod
    def _join(vector: np.ndarray):
        return Board._join_reverse(vector[::-1])[::-1]

    @staticmethod
    def _squeezable(vector: np.ndarray):
        """ If the row vector is filled by all different elements, it is not squeezable. """
        previous = None
        for current in vector:
            if current == 0:
                return True  # Empty rooms exist.
            if current == previous:
                return True  # Two same elements can be joined.
            previous = current
        return False

    def squeezable(self, direction: Literal[0, 90, 180, 270]):
        board = self.rotator.rot(direction).rotate(self.board).copy()
        for i, row in enumerate(board):
            if Board._squeezable(row): return True

        return False 

    @staticmethod
    def _squeeze(vector: np.ndarray):
        """ Squeeze all elements from left to right (0deg). """
        return Board._push(Board._join(vector))

    def squeeze(self, direction: Literal[0, 90, 180, 270]):
        board = self.rotator.rot(direction).rotate(self.board).copy()
        for i, row in enumerate(board):
            board[i, :] = Board._squeeze(row)
        self.board = self.rotator.rot(direction).unrotate(board)

    def available_index(self):
        return [i for i, x in np.ndenumerate(self.board) if int(x) == 0]

    def available_action(self):
        pass

    def step(self, action):

        return self.board, reward, done

    def _random_born(self, num: int = 2):
        try:
            random_index = random.choice(self.available_index())
        except IndexError:
            raise IndexError('The board is full.')
        self.board[random_index] = num

    def is_done(self):
        if not self.available_index():
            for deg in [0,90,180,270]:
                if self.squeezable(deg):
                    return True
        return False


In [713]:
a=np.array([[1,2],[3,4]])
b=np.array([[1,2],[3,4]])
a==b

array([[ True,  True],
       [ True,  True]])

In [714]:
zero_to_31 = [[np.floor((x/2**3)%2),np.floor((x/2**2)%2),np.floor((x/2)%2),x%2] for x in range(2**5)]

In [715]:
Board._squeeze(np.array([2,2,4,4],dtype=np.int32))

array([0, 0, 4, 8])

In [716]:
Board._join_reverse(np.array([2,0,0,2]))

array([0, 0, 0, 4])

In [717]:
b = Board(4)
b.board

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [718]:
b.reset()

for i in range(100):
    b.squeeze(np.random.choice([0,90,180,270]))
    print(b.board)
    b._random_born()
    print(b.board)


    print()

[[0 0 0 0]
 [4 0 0 0]
 [2 0 0 0]
 [0 0 0 0]]
[[0 0 0 0]
 [4 0 2 0]
 [2 0 0 0]
 [0 0 0 0]]

[[4 0 2 0]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[4 0 2 2]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]

[[4 0 2 2]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[4 0 2 2]
 [2 0 0 0]
 [0 0 0 0]
 [0 2 0 0]]

[[4 2 2 2]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[4 2 2 2]
 [2 0 0 2]
 [0 0 0 0]
 [0 0 0 0]]

[[0 0 0 0]
 [0 0 0 0]
 [4 0 0 0]
 [2 2 2 4]]
[[0 0 0 0]
 [0 0 0 2]
 [4 0 0 0]
 [2 2 2 4]]

[[0 0 0 0]
 [0 0 0 0]
 [4 0 0 2]
 [2 2 2 4]]
[[0 0 2 0]
 [0 0 0 0]
 [4 0 0 2]
 [2 2 2 4]]

[[0 0 0 0]
 [0 0 0 0]
 [4 0 0 2]
 [2 2 4 4]]
[[2 0 0 0]
 [0 0 0 0]
 [4 0 0 2]
 [2 2 4 4]]

[[2 0 0 0]
 [0 0 0 0]
 [4 2 0 0]
 [4 8 0 0]]
[[2 0 0 0]
 [0 0 0 0]
 [4 2 0 2]
 [4 8 0 0]]

[[2 0 0 0]
 [0 0 0 0]
 [4 4 0 0]
 [4 8 0 0]]
[[2 2 0 0]
 [0 0 0 0]
 [4 4 0 0]
 [4 8 0 0]]

[[0 0 0 4]
 [0 0 0 0]
 [0 0 0 8]
 [0 0 4 8]]
[[0 0 0 4]
 [0 2 0 0]
 [0 0 0 8]
 [0 0 4 8]]

[[ 0  2  4  4]
 [ 0  0  0 16]
 [ 0  0  0  0]
 [ 0  0  0  0]]
[[ 0  2  4  4]
 [ 0  0  0 16]

IndexError: The board is full.

In [ ]:
b.squeeze(0)

In [ ]:
b.is_done()

True

In [ ]:
b.board

array([[ 2, 16,  8,  2],
       [ 8,  2, 64,  8],
       [ 4, 32,  4, 16],
       [ 2,  8,  2,  2]])

In [ ]:
b.squeeze(0)
b._random_born()
b.board

array([[ 2,  0,  4, 16],
       [ 8, 64, 32,  4],
       [ 2, 16,  2, 32],
       [ 0,  0,  8,  2]])

In [ ]:

b.board

array([[ 2,  0,  4, 16],
       [ 8, 64, 32,  4],
       [ 2, 16,  2, 32],
       [ 0,  0,  8,  2]])

In [ ]:
b._squeeze(np.array([0,0,0,0]))

array([0, 0, 0, 0])